In [2]:
#load csv
import pandas as pd
df_master = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\merged_leads.csv')
df = pd.read_csv(r'C:\Users\AakashAI\Desktop\Repositories\Sales Agent\cleaned_leads.csv')


## Case 1. Generate precise query

In [ ]:
filter_columns = ['Lead ', 'Lead Source', 'Lead Origin', 'Do Not Email','Do Not Call', 'Converted', 'Country','Lead Stage']Source

In [14]:
filter_columns = [
    'Lead Source', 'Lead Origin', 'Do Not Email', 'Do Not Call',
    'Converted', 'Country', 'Lead Stage', 'City'
]

def generate_schema(df, columns):
    schema = []
    for col in columns:
        if col not in df.columns:
            continue
        dtype = str(df[col].dtype)
        examples = df[col].dropna().unique()  # Limit to 5 unique examples
        example_values = ", ".join([str(e) for e in examples])
        schema.append(f"- {col} ({dtype}): e.g., {example_values}")
    return "\n".join(schema)

schema_text = generate_schema(df, filter_columns)

print("Schema for LLM:\n")
print(schema_text)


Schema for LLM:

- Lead Source (object): e.g., Organic Search, Google, Direct Traffic, Olark Chat, Reference, Referral Sites, Welingak Website
- Lead Origin (object): e.g., API, Landing Page Submission, Lead Add Form
- Do Not Email (object): e.g., No, Yes
- Do Not Call (object): e.g., No
- Converted (int64): e.g., 0, 1
- Country (object): e.g., India, United States
- Lead Stage (object): e.g., Unreachable, Not Interested, Qualified, Closed, Interested
- City (object): e.g., Select, Mumbai, Thane & Outskirts, Other Metro Cities, Other Cities of Maharashtra, Other Cities


In [12]:
#find all unique values in each column
unique_values = {col: df[col].unique() for col in df.columns}
unique_values

{'Lead Number': array([660030, 659357, 655510, 655287, 654061, 653724, 651812, 649941,
        648361, 648284, 646302, 643051, 639376, 639288, 639282, 637980,
        636404, 635995, 631411, 631333, 629658, 625880, 624762, 624715,
        624439, 624227, 617845, 617610, 616761, 616058, 615883, 615740,
        615524, 614998, 614523, 613622, 613070, 611277, 608628, 608305,
        606306, 605652, 605267, 605210, 604460, 601100, 598532, 595881,
        595717, 595141, 594521, 593860, 590140, 588689, 588124, 588025,
        582790, 582296, 582094]),
 'Lead Source': array(['Organic Search', 'Google', 'Direct Traffic', 'Olark Chat',
        'Reference', 'Referral Sites', 'Welingak Website'], dtype=object),
 'Lead Origin': array(['API', 'Landing Page Submission', 'Lead Add Form'], dtype=object),
 'Do Not Email': array(['No', 'Yes'], dtype=object),
 'Do Not Call': array(['No'], dtype=object),
 'Converted': array([0, 1]),
 'Country': array(['India', nan, 'United States'], dtype=object),
 'Mobi

In [25]:
from openai import OpenAI
import pandas as pd
client = OpenAI()

def query_dataframe(df, query):
    return 
def nl_to_query(nl_query, df, model="gpt-4o"):
    schema_description = generate_schema(df, filter_columns)
    prompt= f"""
    You are an expert data analyst. I have a pandas DataFrame with these columns:
    {schema_description}

    I want to filter rows based on this user’s query:
    '{nl_query}'

    Please return ONLY a valid Python code snippet that uses pandas to filter the DataFrame.
    The code should be a single line of code that can be executed directly in Python.
    ONLY return the expression that can be used directly inside eval()
     DO NOT include variable assignments like `filtered_df = ...`.\n"
      DO NOT quote the expression.\n"
      Do not add any prefix or suffix to the expression.\n"
    The expression should be a valid pandas DataFrame filtering expression.
    If the query is ambiguous or not feasible, return a simplified expression that best matches the intent.
    """

    messages=[
        {"role": "system", "content": "You are a pandas expert."},
        {"role": "user", "content": prompt}
    ]
    completion = client.chat.completions.create(
            messages=messages,
            model="gpt-4o",
            stream=False,
            # tool_choice="auto",
            temperature=0, # Keep temperature low for deterministic SQL generation
        )
    df_query = completion.choices[0].message.content.strip().replace("\n", " ")

    return df_query
nl_query = "Which leads are not converted yet fromdelhi"
code = nl_to_query(nl_query, df)
print(code)


INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
(df['Converted'] == 0) & (df['City'].str.lower() == 'delhi')


In [27]:
def execute_pandas_query(df, query_str):
    local_vars = {"df": df.copy()}
    try:
        return eval(query_str, {"__builtins__": {}}, local_vars)
    except Exception as e:
        print(f"Error: {e}")
        return None

execute_pandas_query(df, code)

0     False
1     False
2     False
3     False
4     False
5     False
6     False
7     False
8     False
9     False
10    False
11    False
12    False
13    False
14    False
15    False
16    False
17    False
18    False
19    False
20    False
21    False
22    False
23    False
24    False
25    False
26    False
27    False
28    False
29    False
30    False
31    False
32    False
33    False
34    False
35    False
36    False
37    False
38    False
39    False
40    False
41    False
42    False
43    False
44    False
45    False
46    False
47    False
48    False
49    False
50    False
51    False
52    False
53    False
54    False
55    False
56    False
57    False
58    False
dtype: bool

In [29]:
import pandas as pd
from llama_index.experimental.query_engine.pandas import PandasQueryEngine


query_engine = PandasQueryEngine(df=df, verbose=True)

response = query_engine.query("Which leads are not converted yet from delhi?")

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Pandas Instructions:
```
df[(df['Converted'] == 0) & (df['City'] == 'Delhi')]
```
> Pandas Output: Empty DataFrame
Columns: [Lead Number, Lead Source, Lead Origin, Do Not Email, Do Not Call, Converted, Country, Mobile Number, Lead Stage, Lead Grade, Lead Score, Age, Email, City]
Index: []


In [48]:
response

Response(response='Empty DataFrame\nColumns: [Lead Number, Lead Source, Lead Origin, Do Not Email, Do Not Call, Converted, Country, Mobile Number, Lead Stage, Lead Grade, Lead Score, Age, Email, City]\nIndex: []', source_nodes=[], metadata={'pandas_instruction_str': "df[df['Lead Source'] == 'Landing Page Submission']", 'raw_pandas_output': 'Empty DataFrame\nColumns: [Lead Number, Lead Source, Lead Origin, Do Not Email, Do Not Call, Converted, Country, Mobile Number, Lead Stage, Lead Grade, Lead Score, Age, Email, City]\nIndex: []'})

## 2. Generate Hybrid Query

In [40]:
filtered_df = df[df['Lead Origin'] == 'Landing Page Submission']
filtered_df

,Lead Number,Lead Source,Lead Origin,Do Not Email,Do Not Call,Converted,Country,Mobile Number,Lead Stage,Lead Grade,Lead Score,Age,Email,City
1,659357,Google,Landing Page Submission,No,No,0,India,NaN,Unreachable,NaN,120,Select,Yogeshsadarang@yahoo.in,Mumbai
3,655287,Direct Traffic,Landing Page Submission,No,No,0,India,NaN,Not Interested,NaN,90,Select,vkaradkar@in.imshealth.com,Mumbai
4,654061,Direct Traffic,Landing Page Submission,No,No,1,India,NaN,Qualified,B,80,Select,rakshata.nikam@sharekhan.com,Mumbai
8,648361,Referral Sites,Landing Page Submission,No,No,1,India,NaN,Closed,NaN,100,Select,vidula2211@gmail.com,Mumbai
11,643051,Google,Landing Page Submission,No,No,1,India,NaN,Closed,NaN,455,Select,nidhisingh9029291638@gmail.com,Mumbai
12,639376,Google,Landing Page Submission,No,No,1,India,NaN,Qualified,NaN,75,Select,anjalimeena5991@gmail.com,Mumbai
15,637980,Direct Traffic,Landing Page Submission,No,No,1,India,NaN,Qualified,NaN,90,Select,Vaibhavi10rhyme@yahoo.co.in,Mumbai
16,636404,Google,Landing Page Submission,No,No,1,India,NaN,Qualified,B,165,21-25,lalitashetty210992@gmail.com,Thane & Outskirts
21,625880,Direct Traffic,Landing Page Submission,No,No,0,India,NaN,Not Interested,NaN,250,Select,kkpranay@gmail.com,Other Cities of Maharashtra
23,624715,Direct Traffic,Landing Page Submission,No,No,1,India,NaN,Interested,NaN,150,Select,drvrundagb@gmail.com,Thane & Outskirts


In [17]:
def row_to_text(row):
    """
    Combine columns into a single string for embedding. 
    Include relevant free-text fields like university, notes, comments, etc.
    """
    # For the assignment, you might combine relevant columns or ALL columns:
    # We'll just use everything joined by space for demonstration
    row_str = " ".join(str(x) for x in row.values if pd.notna(x))
    return row_str

text_data = [row_to_text(df.iloc[i]) for i in range(len(df))]

In [ ]:
import numpy as np
def get_embedding(text, model="text-embedding-ada-002"):

    response = client.embeddings.create(
        input=text,
        model="text-embedding-3-small"
    )

    return (response.data[0].embedding)
embeddings = [get_embedding(t) for t in text_data]
emb_array = np.array(embeddings, dtype=np.float32)

## Llama Index

In [ ]:
%pip install llama-index-llms-openai llama-index-experimental

In [11]:
df.head()

,city,population,country,area_sq_km
0,London,9000000,UK,1572.0
1,Paris,2141000,France,105.4
2,Tokyo,37435000,Japan,2194.0
3,New York,8399000,USA,783.8
4,Berlin,3645000,Germany,891.8


In [2]:
from openai import OpenAI
import pandas as pd
client = OpenAI()
df = pd.read_csv(r"C:\Users\AakashAI\Desktop\Repositories\Sales Agent\leads_dataset_2.csv")

C:\Users\AakashAI\AppData\Local\Temp\ipykernel_21136\4018573441.py:4: DtypeWarning: Columns (5,44,53,77,97) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r"C:\Users\AakashAI\Desktop\Repositories\Sales Agent\leads_dataset_2.csv")


In [4]:
for column in df.columns:
    print(f"Column: {column}")
    print(f"Data Type: {df[column].dtype}")
    print(f"Unique Values: {df[column].unique()}")

Column: Prospect ID
Data Type: object
Unique Values: ['7927b2df-8bba-4d29-b9a2-b6e0beafe620'
 '2a272436-5132-4136-86fa-dcc88c88f482'
 '8cc8c611-a219-4f35-ad23-fdfd2656bd8a' ...
 'aac550fe-a586-452d-8d3c-f1b62c94e02c'
 '5330a7d1-2f2b-4df4-85d6-64ca2f6b95b9'
 '571b5c8e-a5b2-4d57-8574-f2ffb06fdeff']
Column: Lead Number
Data Type: int64
Unique Values: [660737 660728 660727 ... 579545 579538 579533]
Column: Company
Data Type: object
Unique Values: [nan 'Dr. Ram Manohar Lohia Avadh University /ugc' 'vinod.shah1@gmail.com'
 'poona university' 'mumbai university' 'Yogeshsadarang@yahoo.in'
 'duplicate lead' 'shobit university meerut' 'vinoba bhave university'
 'robbinpandita100@gmail.com' 'amity university'
 'vkaradkar@in.imshealth.com' 'rakshata.nikam@sharekhan.com'
 'smart.ashwin@yahoo.com' 'N.A.' 'nehamenon26@rediffmail.com'
 'lateral student' '9909902397' 'Looking for Friend Sanjay'
 'sankpal1pooja@gmail.com' 'sndt university' 'vidula2211@gmail.com'
 'manish.chand97@Yahoo.com' 'gorakhpur un

In [34]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
)
from llama_index.experimental.query_engine.pandas import (
    PandasInstructionParser,
)
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
instruction_str = (
    "1. Convert the query to executable Python code using Pandas.\n"
    "2. The final line of code should be a Python expression that can be called with the `eval()` function.\n"
    "3. The code should represent a solution to the query.\n"
    "4. PRINT ONLY THE EXPRESSION.\n"
    "5. Do not quote the expression.\n"
)

pandas_prompt_str = (
    "You are working with a pandas dataframe in Python.\n"
    "The name of the dataframe is `df`.\n"
    "This is the result of `print(df.head())`:\n"
    "{df_str}\n\n"
    "Follow these instructions:\n"
    "{instruction_str}\n"
    "Query: {query_str}\n\n"
    "Expression:"
)
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n\n"
    "Pandas Instructions (optional):\n{pandas_instructions}\n\n"
    "Pandas Output: {pandas_output}\n\n"
    "Response: "
)

pandas_prompt = PromptTemplate(pandas_prompt_str).partial_format(
    instruction_str=instruction_str, df_str=df.head(5)
)
pandas_output_parser = PandasInstructionParser(df)
response_synthesis_prompt = PromptTemplate(response_synthesis_prompt_str)
llm = OpenAI(model="gpt-4o")
qp = QP(
    modules={
        "input": InputComponent(),
        "pandas_prompt": pandas_prompt,
        "llm1": llm,
        "pandas_output_parser": pandas_output_parser,
        "response_synthesis_prompt": response_synthesis_prompt,
        "llm2": llm,
    },
    verbose=True,
)
qp.add_chain(["input", "pandas_prompt", "llm1", "pandas_output_parser"])
qp.add_links(
    [
        Link("input", "response_synthesis_prompt", dest_key="query_str"),
        Link(
            "llm1", "response_synthesis_prompt", dest_key="pandas_instructions"
        ),
        Link(
            "pandas_output_parser",
            "response_synthesis_prompt",
            dest_key="pandas_output",
        ),
    ]
)
# add link from response synthesis prompt to llm2
qp.add_link("response_synthesis_prompt", "llm2")
response = qp.run(
    query_str="List users that came from organic leads from delhi ",
)

> Running module input with input: 
query_str: List users that came from organic leads from delhi 

> Running module pandas_prompt with input: 
query_str: List users that came from organic leads from delhi 

> Running module llm1 with input: 
messages: You are working with a pandas dataframe in Python.
The name of the dataframe is `df`.
This is the result of `print(df.head())`:
   Lead Number     Lead Source              Lead Origin Do Not Email  \
...

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
> Running module pandas_output_parser with input: 
input: assistant: df[(df['Lead Source'] == 'Organic Search') & (df['City'] == 'Delhi')]

> Running module response_synthesis_prompt with input: 
query_str: List users that came from organic leads from delhi 
pandas_instructions: assistant: df[(df['Lead Source'] == 'Organic Search') & (df['City'] == 'Delhi')]
pandas_out

In [35]:
response

ChatResponse(message=ChatMessage(role=<MessageRole.ASSISTANT: 'assistant'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='It appears that there are no users in the dataset who came from organic leads originating from Delhi. The query returned an empty result, indicating that there are no entries matching these specific criteria.')]), raw=ChatCompletion(id='chatcmpl-BLQB0AC3o0Uwtu3SjCpuDZAWfOMcq', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='It appears that there are no users in the dataset who came from organic leads originating from Delhi. The query returned an empty result, indicating that there are no entries matching these specific criteria.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1744445210, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_92f14e8683', usage=CompletionUsage(completion_tok

## Embeddings

In [13]:
%pip install sentence-transformers

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import os
import pandas as pd
import numpy as np
import pickle
from sentence_transformers import SentenceTransformer
from scipy.spatial.distance import cdist # For searching

# --- Configuration ---
DATA_DIR = "./data"
# *** YOUR LEAD DATA CSV FILE NAME ***
LEAD_FILE_NAME = "merged_leads.csv"
# *** NAME FOR THE GENERATED EMBEDDING FILE ***
EMBEDDING_FILE_NAME = "lead_all_columns_textualized_embeddings.pk"
MODEL_PATH = "./embedding_model"  # Or "all-mpnet-base-v2" if not saved locally
TOP_N_RESULTS = 10

# --- Columns to Textualize (Example Subset) ---
# Adjust this list based on what you deem important for similarity
COLUMNS_FOR_DESCRIPTION = [
    'Lead Origin_x', 'Lead Source_x', 'Converted', 'TotalVisits_x',
    'Total Time Spent on Website', 'Last Activity_x', 'Specialization_x',
    'What is your current occupation_x', 'What matters most to you in choosing a course',
    'Tags_x', 'Lead Quality_x', 'Lead Profile_x', 'City',
    'Asymmetrique Activity Score_x', 'Asymmetrique Profile Score_x',
    'A free copy of Mastering The Interview', #'Notes' # Add 'Notes' if you want to include it
]

# --- Function to Create Lead Description ---
def generate_lead_description(row, columns_to_use):
    description_parts = []
    for col in columns_to_use:
        value = row.get(col) # Use .get() for safety if a column was missed in selection
        if pd.isna(value) or str(value).strip() == "" or str(value).lower() == 'select':
            # Skip empty, NaN, or generic "Select" values
            continue

        # Clean column name for display
        col_name_clean = col.replace('_x', '').replace('_y', '').replace('_', ' ').title()

        # Textualize based on type (simple example)
        if isinstance(value, bool) or str(value).lower() in ['yes', 'no']:
             bool_val = str(value).lower() == 'yes' or str(value) == '1' or value is True
             description_parts.append(f"{col_name_clean}: {'Yes' if bool_val else 'No'}.")
        elif isinstance(value, (int, float)):
             description_parts.append(f"{col_name_clean}: {value}.")
        else: # Assume text/categorical
             description_parts.append(f"{col_name_clean}: {str(value).strip()}.") # Ensure string and strip whitespace

    # Combine parts into a single description
    full_description = " ".join(description_parts)
    # Add Notes at the end if present and chosen
    if 'Notes' in columns_to_use and pd.notna(row.get('Notes')):
         full_description += f" Notes: {str(row.get('Notes')).strip()}"

    return full_description if full_description else "No details provided." # Fallback for empty rows


# --- 1. Load Lead Data ---
data_file_path = os.path.join(DATA_DIR, LEAD_FILE_NAME)
if not os.path.exists(data_file_path):
    raise FileNotFoundError(f"Lead data file not found: {data_file_path}")

try:
    raw_df = pd.read_csv(data_file_path)
    print(f"Loaded lead data: {raw_df.shape[0]} rows, {raw_df.shape[1]} columns.")
except UnicodeDecodeError:
    print("Trying with 'ISO-8859-1' encoding.")
    raw_df = pd.read_csv(data_file_path, encoding='ISO-8859-1')
except Exception as e:
    raise RuntimeError(f"Error loading CSV: {e}")

# --- Verify required columns exist ---
missing_cols = [col for col in COLUMNS_FOR_DESCRIPTION if col not in raw_df.columns]
if missing_cols:
    raise ValueError(f"Missing columns needed for description: {missing_cols}")
if 'Prospect ID' not in raw_df.columns: # Need an ID column
     raise ValueError("Missing 'Prospect ID' column for identifying leads.")


# --- 2. Load Embedding Model ---
# (Same logic as before: loads or downloads/saves)
if not os.path.exists(MODEL_PATH):
     print(f"Local model path '{MODEL_PATH}' not found. Loading 'all-mpnet-base-v2'.")
     try:
         embedding_model = SentenceTransformer("all-mpnet-base-v2")
         print("Saving model to './embedding_model'.")
         os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
         embedding_model.save(MODEL_PATH)
     except Exception as e:
         raise RuntimeError(f"Could not load/save SentenceTransformer model. Error: {e}")
else:
    print(f"\n--- Loading Embedding Model from {MODEL_PATH} ---")
    embedding_model = SentenceTransformer(MODEL_PATH)


# --- 3. Generate/Load Embeddings for Textualized Descriptions ---
embedding_file_path = os.path.join(DATA_DIR, EMBEDDING_FILE_NAME)
print(f"\n--- Checking for Lead Embeddings at {embedding_file_path} ---")

if not os.path.exists(embedding_file_path):
    print("Embedding file not found. Generating textual descriptions and embeddings...")
    print("This might take a while...")

    # --- Create Textual Descriptions ---
    print("Generating descriptions...")
    raw_df['text_description'] = raw_df.apply(
        lambda row: generate_lead_description(row, COLUMNS_FOR_DESCRIPTION),
        axis=1
    )
    print("Sample Description:")
    print(raw_df['text_description'].iloc[0]) # Show one example

    # Ensure the data directory exists
    os.makedirs(os.path.dirname(embedding_file_path), exist_ok=True)

    # --- Encode Descriptions ---
    print("Generating embeddings...")
    lead_embeddings = embedding_model.encode(
        raw_df['text_description'].values,
        show_progress_bar=True
    )
    print(f"Generated lead embeddings with shape: {lead_embeddings.shape}")

    # --- Save Embeddings ---
    try:
        with open(embedding_file_path, "wb") as f:
            pickle.dump(lead_embeddings, f)
        print(f"Successfully saved lead embeddings to {embedding_file_path}")
    except Exception as e:
        print(f"Error saving embedding file {embedding_file_path}. Error: {e}")
else:
    print(f"Loading existing lead embeddings from {embedding_file_path}")
    try:
        with open(embedding_file_path, "rb") as f:
            lead_embeddings = pickle.load(f)
        print(f"Loaded embeddings with shape: {lead_embeddings.shape}")
        # Verification
        if len(lead_embeddings) != len(raw_df):
             raise ValueError(f"Mismatch! Embeddings ({len(lead_embeddings)}) vs DataFrame rows ({len(raw_df)}).")
        # Generate descriptions even if embeddings loaded, needed for display
        print("Generating descriptions for display...")
        raw_df['text_description'] = raw_df.apply(
            lambda row: generate_lead_description(row, COLUMNS_FOR_DESCRIPTION),
            axis=1
        )
    except Exception as e:
        raise RuntimeError(f"Error loading embedding file {embedding_file_path}. Error: {e}")

# Ensure embeddings are numpy array
lead_embeddings = np.array(lead_embeddings)
# Keep track of IDs in the same order
prospect_ids = raw_df["Prospect ID"].values

# --- 4. Perform Similarity Search ---
# *** MODIFY QUERY to reflect the kind of lead you are looking for ***
# Example: Find unemployed leads interested in Marketing, with high profile score, who want better prospects
query_text = "Unemployed lead interested in marketing management with high profile score looking for better career prospects"
print(f"\n--- Searching for leads similar to: '{query_text}' ---")

# Encode the query
query_emb = embedding_model.encode(query_text)

# Calculate L2 (Euclidean) distances
query_emb_reshaped = query_emb.reshape(1, -1)
distances = cdist(query_emb_reshaped, lead_embeddings, metric='euclidean')[0]

# Get the indices of the top N results
sorted_indices = np.argsort(distances)
top_n_indices = sorted_indices[:TOP_N_RESULTS]

# Get the corresponding ProspectIDs
top_n_prospect_ids = prospect_ids[top_n_indices]

print(f"\n--- Top {TOP_N_RESULTS} Prospect IDs ---")
print(top_n_prospect_ids.tolist())

# --- 5. Retrieve Details for Top Results ---
print("\n--- Details of Top Matching Leads ---")
# Get the full rows for the top prospect IDs
results_df = raw_df[raw_df["Prospect ID"].isin(top_n_prospect_ids)].copy()

# Order results based on similarity score
results_df['Prospect ID'] = pd.Categorical(results_df['Prospect ID'], categories=top_n_prospect_ids.tolist(), ordered=True)
results_df = results_df.sort_values('Prospect ID')

# Display selected info + the generated description
columns_to_display = ['Prospect ID', 'Converted', 'Lead Source_x', 'Specialization_x', 'What is your current occupation_x', 'Total Time Spent on Website', 'Lead Quality_x', 'text_description']
# Filter display columns to only those that actually exist in the dataframe
columns_to_display = [col for col in columns_to_display if col in results_df.columns]

print(results_df[columns_to_display].to_string(index=False))


print("\n--- Script Finished ---")

Loaded lead data: 59 rows, 159 columns.
Local model path './embedding_model' not found. Loading 'all-mpnet-base-v2'.
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: all-mpnet-base-v2
Load pretrained SentenceTransformer: all-mpnet-base-v2
Saving model to './embedding_model'.
INFO:sentence_transformers.SentenceTransformer:Save model to ./embedding_model
Save model to ./embedding_model

--- Checking for Lead Embeddings at ./data\lead_all_columns_textualized_embeddings.pk ---
Embedding file not found. Generating textual descriptions and embeddings...
This might take a while...
Generating descriptions...
Sample Description:
Lead Origin: API. Lead Source: Organic Search. Converted: 0. Totalvisits: 2.0. Total Time Spent On Website: 12. Last Activity: Email Opened. What Is Your Current Occupation: Unemployed. What Matters Most To You In Choosing A Course:

Batches: 100%|██████████| 2/2 [00:04<00:00,  2.42s/it]


Generated lead embeddings with shape: (59, 768)
Successfully saved lead embeddings to ./data\lead_all_columns_textualized_embeddings.pk

--- Searching for leads similar to: 'Unemployed lead interested in marketing management with high profile score looking for better career prospects' ---


Batches: 100%|██████████| 1/1 [00:00<00:00, 21.90it/s]


--- Top 10 Prospect IDs ---
['7b27d688-8eef-4805-a3ed-fb3c678cd92c', '59e989b8-8d43-48be-90cd-1d779e7fe2be', 'c5dd06bc-4117-4a1f-9eef-f2a1cfaa679e', '84feb092-879e-4662-a10d-8c9f62dde1a8', '79d55284-fddc-431e-8681-6a666f133dd4', '94b60093-49cd-41f5-be6e-ebc3c009ae54', '4564e39b-c8e4-4583-b29e-91b2108c08bf', 'd8c16d3c-0bdb-4b59-873a-1370165fc885', '8b6918d7-20da-49f3-bd4e-91b67fb09a4a', '92b27be3-3150-4604-b38a-ab0077a9bbe2']

--- Details of Top Matching Leads ---
                         Prospect ID  Converted  Lead Source_x          Specialization_x What is your current occupation_x  Total Time Spent on Website    Lead Quality_x                                                                                                                                                                                                                                                                                                                                                                          

In [23]:
# --- 4. Perform Similarity Search ---
# *** MODIFY QUERY to reflect the kind of lead you are looking for ***
# Example: Find unemployed leads interested in Marketing, with high profile score, who want better prospects
query_text = "Leads from landing page" # <--- CHANGE THIS LINE
print(f"\n--- Searching for leads similar to: '{query_text}' ---")

# Encode the query
query_emb = embedding_model.encode(query_text)
# ... (rest of the search code: distance calculation, sorting, retrieving results) ...


--- Searching for leads similar to: 'Leads from landing page' ---


Batches: 100%|██████████| 1/1 [00:00<00:00, 17.79it/s]


In [32]:
query_text = "Lead Origin_x Landing Page Submission"
print(f"\n--- Searching for leads similar to: '{query_text}' ---")

# --- Encode the Query ---
query_emb = embedding_model.encode(query_text)

# --- Calculate Distances ---
# Reshape query embedding for cdist
query_emb_reshaped = query_emb.reshape(1, -1)
# Calculate Euclidean distance between the query and all lead description embeddings
print("Calculating distances...")
distances = cdist(query_emb_reshaped, lead_embeddings, metric='euclidean')[0]
print("Distance calculation complete.")

# --- Find Nearest Neighbors ---
# Get the indices of the leads sorted by distance (ascending - closest first)
print("Sorting results...")
sorted_indices = np.argsort(distances)

# Get the indices and IDs of the top N results
top_n_indices = sorted_indices[:TOP_N_RESULTS]
top_n_prospect_ids = prospect_ids[top_n_indices]

# (Optional) Get the distances for the top results
top_n_distances = distances[top_n_indices]

print(f"\n--- Top {TOP_N_RESULTS} Prospect IDs (Closest First) ---")
print(top_n_prospect_ids.tolist())
# print("\n--- Corresponding Distances (Lower is better) ---")
# print(top_n_distances.tolist())


# ================================================================
# --- 5. Retrieve and Display Details for Top Results ---
# ================================================================
print("\n--- Details of Top Matching Leads (Ordered by Similarity) ---")

# Filter the original DataFrame to get the full rows for the top prospect IDs
results_df = raw_df[raw_df["Prospect ID"].isin(top_n_prospect_ids)].copy()

# --- Order the results based on the similarity search order ---
# Convert 'Prospect ID' to a categorical type with the order defined by search results
results_df['Prospect ID'] = pd.Categorical(
    results_df['Prospect ID'],
    categories=top_n_prospect_ids.tolist(), # Use the order found by argsort
    ordered=True
)
# Sort the DataFrame based on this categorical order
results_df = results_df.sort_values('Prospect ID')

# --- Select columns to display ---
# Choose columns you want to see in the output table
columns_to_display = [
    'Prospect ID',
    'Converted',
    'Lead Origin_x',
    'Lead Source_x',
    'Specialization_x',
    'What is your current occupation_x',
    'Total Time Spent on Website',
    'Lead Quality_x',
    'City',
    'Extracted Email'
    'text_description' # Include the generated description for context
    ]
# Filter display columns to only those that actually exist in the dataframe
columns_to_display = [col for col in columns_to_display if col in results_df.columns]

# --- Print the results ---
# Use to_string to display more content if needed, or just print(results_df[columns_to_display])
print(results_df[columns_to_display].to_string(index=False))




--- Searching for leads similar to: 'Lead Origin_x Landing Page Submission' ---


Batches: 100%|██████████| 1/1 [00:00<00:00, 27.45it/s]

Calculating distances...
Distance calculation complete.
Sorting results...

--- Top 10 Prospect IDs (Closest First) ---
['da76e32e-f09b-4830-8d7f-a4996d6125df', 'f12dfdf7-c8a6-4765-a681-dde85cf60453', 'd23ae8c1-7938-4a8b-b22f-459adf894488', '94b60093-49cd-41f5-be6e-ebc3c009ae54', '92b27be3-3150-4604-b38a-ab0077a9bbe2', '0470e536-1d51-4dbb-bbc6-565f2536a7c6', 'bad2625c-31f7-48da-bb7d-812c08673014', '60439638-e8e3-4494-8d09-90f545b0ed85', '8136361c-5fcb-4213-832d-5f0e488af8a2', '1663c16e-0244-462a-bd1c-695d869b6d12']

--- Details of Top Matching Leads (Ordered by Similarity) ---
                         Prospect ID  Converted           Lead Origin_x  Lead Source_x                  Specialization_x What is your current occupation_x  Total Time Spent on Website    Lead Quality_x               City
da76e32e-f09b-4830-8d7f-a4996d6125df          1           Lead Add Form      Reference              Marketing Management                        Unemployed                            0          Mi

In [ ]:

# --- 6. Retrieve Product Names for Top Results ---
print("\n--- Top Matching Product Names ---")
# Filter the original DataFrame to get the names for the top product IDs
results_df = raw_df.loc[raw_df["ProductID"].isin(top_n_product_ids), ["ProductID", "ProductName"]]

# To maintain the order from the similarity search:
# Convert top_n_product_ids to a list to use it for ordering
ordered_ids = top_n_product_ids.tolist()
# Set ProductID as index for easy reordering based on the search results
results_df = results_df.set_index('ProductID')
# Reindex based on the order found by the search
ordered_results_df = results_df.reindex(ordered_ids)

# Print the ordered results
for pid, row in ordered_results_df.iterrows():
    print(f"  - {row['ProductName']} (ID: {pid})") # Changed formatting slightly

print("\n--- Script Finished ---")